In [ ]:
!sudo pip3 install keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow opencv--python sklearn optuna scikit-image ray ray[tune]

In [1]:
import pickle
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import asarray
from numpy import save

with open('training.pkl', 'rb') as f:
     train = pickle.load(f)
    
with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)
        
with open('validation.pkl','rb') as f:
     val = pickle.load(f)
        
train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

model= load_model("model.h5")


Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [10]:

import shutil
import tempfile

import tensorflow as tf
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop
# import tensorflow_datasets as tfds

import ray 
from ray import tune

In [11]:
CLASSES = 2
def create_model(learning_rate):
    assert learning_rate > 0, "Did you set the right configuration?"
    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(CLASSES, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False
    
    optimizer = RMSprop(lr=learning_rate)
    model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
def train():
    train_x, train_y, val_x, val_y = train_photos,train_labels,val_photos,val_labels
    model = create_model(learning_rate=0.1)
    checkpoint_callback = ModelCheckpoint(
        "model.h5", monitor='accuracy', save_best_only=True)
    model.fit(
        train_x, train_y, 
        validation_data=(val_x, val_y),
        verbose=1, batch_size=2, epochs=1, callbacks=[checkpoint_callback])
    return model

In [13]:
original_model = train()
test_x, test_y = test_photos.astype("float32"), test_labels
original_loss, original_accuracy = original_model.evaluate(test_x, test_y, batch_size=2)
print("Loss is {:0.4f}".format(original_loss))
print("Accuracy is {:0.4f}".format(original_accuracy))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


Train on 432 samples, validate on 48 samples
Epoch 1/1
120/120 [==============================] - 16s 134ms/step
Loss is 8.0590
Accuracy is 0.5000


In [14]:
import tensorflow.keras as keras
from ray import tune
class TuneReporterCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        tune.report(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))

In [15]:
def tune():  # TODO: Change me.
    train_x, train_y, val_x, val_y = train_photos,train_labels,val_photos,val_labels
    model = create_model(learning_rate=0.1)
    
    model.fit(
        train_x, train_y, 
        validation_data=(val_x, val_y),
        verbose=0, 
        batch_size=2, 
        epochs=1, 
        callbacks=callbacks)

In [ ]:
import numpy as np 
hyperparameter_space = {"lr": tune.loguniform(0.001, 0.1)}  # TODO: Fill me out.
num_samples = 10
HP_KEYS = ["lr"]

ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
ray.init(log_to_driver=False)
!rm -rf ~/ray_results/tune

analysis = tune.run(
    tune_iris, 
    verbose=1, 
    config=hyperparameter_space)

assert len(analysis.trials) == 10, "Did you set the correct number of samples?"

2020-08-13 10:31:16,530	INFO resource_spec.py:231 -- Starting Ray with 0.44 GiB memory available for workers and up to 0.23 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-13 10:31:17,292	INFO services.py:1193 -- View the Ray dashboard at localhost:8265
2020-08-13 10:31:17,301	WARNING services.py:1567 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.
2020-08-13 10:31:17,335	WARNING services.py:1567 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files 